In [1]:
# import course;course.header()
import pandas as pd
from datetime import date
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
pio.renderers.default = "browser"
import numpy as np
import plotly.express as px
import re
import itertools
from collections import deque

In [ ]:
"hello"

# Retrieve COVID data from ECDC 

In [2]:
covid_url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/json/"
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import json
import urllib

In [3]:
covid_json_unformated = urllib.request.urlopen(covid_url).read().decode("utf-8")
covid_json = json.loads(covid_json_unformated)
cdf = pd.DataFrame(covid_json['records'])

Rename columns to something more Pythonian. If you think they look already great, then at least rename `notification_rate_per_100000_population_14-days` to `14d-incidence`

Identify which columns have not been casted to an appropriate type during loading!

We did not cover datetime objects in pandas, however they are quite powerful!

Try:

In [4]:
cdf['date_reported'] =  pd.to_datetime(cdf['dateRep'])

Now you can treat the column as a datetime objects using `df[col].dt` , e.g. https://docs.python.org/3/library/datetime.html#datetime.date.year

Create a new column `deltaTime_since_start_of_recording`

Create histograms for different columns or describe the df. Can you spot the inconsistency in the data? Fix it! :)

Identify those countries (grouped by continent) which showed the most drastic increase most drastic and decrease of the `14d-incidence` within the different years since recording. Visualize intuitively!

Which country showed the highest/lowest fluctuation in `14d-incidence` within a year?

Create a line plot showing the `14-incidence` for all European countries. Use `groupby` operation to generate the data list for the plotly plot. 

Create a smoothed version of the `14d-incidence` by averaging 3 months.

Create a radial plot of death rate / 100000 people (see popData2019), where one year completes a circle, i.e. 360˚. Visualize the recored years for Italy, Germany, Sweden and Greece. Hint you might need to turn the dateTime into `day within the year` (%j) and adjust 365 to 360 degrees. 

Optional: Find "regular" mortality rates for those countries and visualize it in the plot as well.

In [5]:
cdf.head(5)

,dateRep,year_week,cases_weekly,deaths_weekly,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,notification_rate_per_100000_population_14-days,date_reported
0,08/02/2021,2021-05,238,8,Afghanistan,AF,AFG,38041757.0,Asia,1.33,2021-08-02
1,01/02/2021,2021-04,267,16,Afghanistan,AF,AFG,38041757.0,Asia,2.58,2021-01-02
2,25/01/2021,2021-03,713,43,Afghanistan,AF,AFG,38041757.0,Asia,3.34,2021-01-25
3,18/01/2021,2021-02,557,45,Afghanistan,AF,AFG,38041757.0,Asia,3.24,2021-01-18
4,11/01/2021,2021-01,675,71,Afghanistan,AF,AFG,38041757.0,Asia,4.15,2021-11-01


In [6]:
wcdf = cdf.copy()

In [224]:
def clean_data(wcdf):
    wcdf.rename(columns=
            {'notification_rate_per_100000_population_14-days':'14d-incidence',
                'countriesAndTerritories': 'region',
                'geoId': 'geo_id',
                'countryterritoryCode': 'r_code',
                'popData2019': 'population',
                'continentExp': 'continent'},
            inplace=True)

    wcdf['date_reported'] =  pd.to_datetime(wcdf['dateRep'], format= '%d/%m/%Y')
    wcdf["14d-incidence"] = pd.to_numeric(wcdf["14d-incidence"])

    today = date.today()
    date_today = today.strftime("%m-%d-%Y")
    wcdf["time_recordings"] = (pd.to_datetime(date_today) - wcdf["date_reported"])
    
    wcdf.dropna(inplace=True)
    wcdf = wcdf[wcdf.cases_weekly >= 0]
    wcdf = wcdf[wcdf.deaths_weekly >= 0]
    wcdf = wcdf[wcdf.population >= 0]
    wcdf = wcdf[wcdf["14d-incidence"] >= 0]
    
    return wcdf

In [225]:
wcdf = clean_data(cdf)

In [226]:
wcdf[time_recordings]

,dateRep,year_week,cases_weekly,deaths_weekly,region,geo_id,r_code,population,continent,14d-incidence,date_reported,time_recordings
0,08/02/2021,2021-05,238,8,Afghanistan,AF,AFG,38041757.0,Asia,1.33,2021-02-08,6 days
1,01/02/2021,2021-04,267,16,Afghanistan,AF,AFG,38041757.0,Asia,2.58,2021-02-01,13 days
2,25/01/2021,2021-03,713,43,Afghanistan,AF,AFG,38041757.0,Asia,3.34,2021-01-25,20 days
3,18/01/2021,2021-02,557,45,Afghanistan,AF,AFG,38041757.0,Asia,3.24,2021-01-18,27 days
4,11/01/2021,2021-01,675,71,Afghanistan,AF,AFG,38041757.0,Asia,4.15,2021-01-11,34 days
...,...,...,...,...,...,...,...,...,...,...,...,...
10641,27/04/2020,2020-17,6,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.12,2020-04-27,293 days
10642,20/04/2020,2020-16,11,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.11,2020-04-20,300 days
10643,13/04/2020,2020-15,5,2,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05,2020-04-13,307 days
10644,06/04/2020,2020-14,2,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05,2020-04-06,314 days


In [9]:
def create_hist(df, column):
    data = [
        go.Histogram(
            x=df[column]
        )
    ]
    fig = go.Figure(data=data)
    fig.show()
    return

In [10]:
create_hist(wcdf, 'cases_weekly')

In [9]:
def sort_dataframe(dataframe, top_group, minor_group, sorted_col):
    sorted_df = dataframe\
        .sort_values([top_group])\
        .groupby([minor_group])\
        .apply(lambda x: x.sort_values([sorted_col], ascending=True))\
        .reset_index(drop=True)\
        .groupby(top_group)
    
    sorted_dict = {}
    
    output_sorted_by = list(np.unique(wcdf[top_group]))
    
    for continent in output_sorted_by:
        sorted_dict[continent] = sorted_df.get_group(continent)
    
    return sorted_dict, output_sorted_by

In [10]:
sort_dataframe, list_sorted_by = sort_dataframe(wcdf, 'continent', 'region', 'time_recordings')

In [100]:
def cal_all(sort_dataframe, continent_list):

    for pos, continent in enumerate(continent_list):
        strongest_increase = []
        srongest_decrease = []
        flux = []
        df = sort_dataframe[continent].reset_index(drop=True)
        continent_countries = list(np.unique(df['region']))
        for country in continent_countries:
            strongest_increase.append(df.groupby('region').get_group(country)['14d-incidence'].diff().max())
            srongest_decrease.append(df.groupby('region').get_group(country)['14d-incidence'].diff().min())
            flux.append(df.groupby('region').get_group(country)['14d-incidence'].diff().std())

        data = {
            "strongest_increase": strongest_increase,
            "strongest_decrease": srongest_decrease,
            "flux": flux,
            "country": continent_countries,
            "continent": [continent] * len(continent_countries)
        }        
        if pos == 0:
            comb_df = pd.DataFrame(data)

        if pos > 0:
            df = pd.DataFrame(data)
            comb_df = pd.concat([comb_df, df])
    
    return comb_df

In [127]:
comb_df = cal_all(sort_dataframe, ['Africa', 'America', 'Asia', 'Europe', 'Oceania'])

In [254]:
def get_strongest_inc_dec_flux(dataframe, continent_list, all_df):
    
    continent_countries = list(dataframe['country'])
    for pos, continent in enumerate(continent_list):
        gr_cont = dataframe.groupby("continent").get_group(continent)
    
        id_inc = gr_cont['strongest_increase'].idxmax()
        inc_val = gr_cont['strongest_increase'].iloc[id_inc]
        inc_name = gr_cont['country'].iloc[id_inc]

        id_dec = gr_cont['strongest_decrease'].idxmin()
        dec_val = gr_cont['strongest_decrease'].iloc[id_dec]
        dec_name = gr_cont['country'].iloc[id_dec]

        id_flux = gr_cont['flux'].idxmax()
        flux_val = gr_cont['flux'].iloc[id_flux]
        flux_name = gr_cont['country'].iloc[id_flux]
    
        print("The str. increase for " + str(continent) + " in the 14d-inc. was by " + inc_name + " with an inc of " + \
         str(round(inc_val,2)))
        print("The str. decrease for " + str(continent) + " in the 14d-inc. was by " + dec_name + " with an dec of " + \
         str(round(-dec_val,2)))
        print("The str. flux for " + str(continent) + " in the 14d-inc. was by " + flux_name + " with a STD of " + \
         str(round(flux_val,2)) + '\n')
        
        d_i=[]
        d_d=[]
        d_f = []

        for times in all_df[continent].groupby('region').get_group(inc_name)["time_recordings"]:
            d_i.append(times.days)
        for times in all_df[continent].groupby('region').get_group(dec_name)["time_recordings"]:
            d_d.append(times.days)
        for times in all_df[continent].groupby('region').get_group(flux_name)["time_recordings"]:
            d_f.append(times.days)
        
        df_inc_ = {
            "country" : inc_name,
            "14d-incidence" : all_df[continent].groupby("region").get_group(inc_name)['14d-incidence'],
            "days ago" : d_i[::-1]
        }
        df_dec_ = {
            "country" : inc_name,
            "14d-incidence" : all_df[continent].groupby("region").get_group(dec_name)['14d-incidence'],
            "days ago" : d_d[::-1]
        }
        df_flux_ = {
            "country" : flux_name,
            "14d-incidence" : all_df[continent].groupby("region").get_group(flux_name)['14d-incidence'],
            "days ago" : d_f[::-1]
        }
        if pos == 0:
            df_inc = pd.DataFrame(df_inc_)
            df_dec = pd.DataFrame(df_dec_)
            df_flux = pd.DataFrame(df_flux_)

        if pos > 0:
            df_0 = pd.DataFrame(df_inc_)
            df_1 = pd.DataFrame(df_dec_)
            df_2 = pd.DataFrame(df_flux_)
            
            df_inc = pd.concat([df_inc, df_0])
            df_dec = pd.concat([df_dec, df_1])
            df_flux = pd.concat([df_flux, df_2])
        
    return df_inc.reset_index(drop=True), df_dec.reset_index(drop=True), df_flux.reset_index(drop=True)
    

In [255]:
df_inc, df_dec, df_flux = get_strongest_inc_dec_flux(comb_df,  ['Africa', 'America', 'Asia', 'Europe', 'Oceania'],sort_dataframe)

The str. increase for Africa in the 14d-inc. was by Eswatini with an inc of 153.99
The str. decrease for Africa in the 14d-inc. was by Seychelles with an dec of 227.13
The str. flux for Africa in the 14d-inc. was by Seychelles with a STD of 47.4

The str. increase for America in the 14d-inc. was by Belize with an inc of 301.27
The str. decrease for America in the 14d-inc. was by Aruba with an dec of 504.18
The str. flux for America in the 14d-inc. was by Aruba with a STD of 139.97

The str. increase for Asia in the 14d-inc. was by Israel with an inc of 294.99
The str. decrease for Asia in the 14d-inc. was by Israel with an dec of 296.37
The str. flux for Asia in the 14d-inc. was by Israel with a STD of 100.14

The str. increase for Europe in the 14d-inc. was by Holy_See with an inc of 1717.79
The str. decrease for Europe in the 14d-inc. was by Holy_See with an dec of 1717.79
The str. flux for Europe in the 14d-inc. was by Gibraltar with a STD of 456.32

The str. increase for Oceania in

In [257]:
plot_linear(df_inc, "contries and their strongest increase", "days ago", "14d-incidence", "country" )

In [258]:
plot_linear(df_dec, "contries and their strongest decrease", "days ago", "14d-incidence", "country" )

In [259]:
plot_linear(df_flux, "contries and their strongest flux", "days ago", "14d-incidence", "country" )

In [192]:
def get_continent_df(continent, full_df, year_to_remove):
    cont_df = sort_dataframe[continent].reset_index(drop=True)
    count_df_yr = cont_df[~cont_df.year_week.str.contains(year_to_remove)].reset_index(drop=True)
    
    Europe_countries = list(np.unique(count_df_yr['region']))
    week_nr = []
    week = []

    for i in range(1,53):
        week_nr.append('week_' + str(i).zfill(2))

    for country in Europe_countries:
        weeks_in_total = len(count_df_yr.groupby('region').get_group(country)['year_week'])
        week.append(week_nr[::-1][:weeks_in_total])

    week_number = list(itertools.chain.from_iterable(week))
    count_df_yr['week_number'] = week_number

    return count_df_yr


In [193]:
Europe2020 = get_continent_df("Europe", sort_dataframe, "2021")

In [198]:
def plot_linear(dataframe, title, x, y, color):
    fig = px.line(dataframe[::-1], y=y, x=x, color=color,
                    template="plotly_dark",
                     title = title)
    fig.show()

def plot_circ(dataframe, title, r, theta, color, adj_range):
    fig = px.line_polar(dataframe, r=r, theta=theta, color=color, line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,range_theta=adj_range,
                    template="plotly_dark",
                        title = title)
    fig.show()

In [22]:
plot_circ(Europe2020,"14 day incidence in Europe per Week", "14d-incidence","week_number","region", [0,360])

In [23]:
plot_linear(Europe2020,"14 day incidence in Europe per Week", "week_number", "14d-incidence", "region")

In [24]:
def smoothend_incidence(continent_df, window_length_days):

    avg_indc = []
    continent_countries = list(np.unique(continent_df['region']))
    
    for country in continent_countries:
        window_entries = deque([], window_length_days)
        df = continent_df[['region', 'week_number', '14d-incidence']].groupby('region').get_group(country).reset_index(drop=True)
        for incidence in df['14d-incidence']:
            window_entries.append(incidence)
            avg_indc.append(sum(window_entries) /len(window_entries))

    continent_df['avg_incidence'] = avg_indc
    return continent_df

In [25]:
Europe2020_3 = smoothend_incidence(Europe2020, 3)

In [26]:
plot_linear(Europe2020_3,"smoothend incidence rate over 3 weeks", "week_number", "avg_incidence", "region")

In [200]:
df_inc

,country,14d,date
0,Eswatini,283.94,2021-05
1,Eswatini,249.10,2021-04
2,Eswatini,107.39,2021-03
3,Eswatini,261.38,2021-02
4,Eswatini,263.38,2021-01
...,...,...,...
237,French_Polynesia,1.43,2020-17
238,French_Polynesia,5.01,2020-16
239,French_Polynesia,6.45,2020-15
240,French_Polynesia,8.24,2020-14


In [190]:
def get_death_rate_per_inh(dataframe, list_of_countries, nr_people):
    for pos, country in enumerate(list_of_countries):
        week_nr = []
        week_nr.append(dataframe.groupby('region').get_group(country)['week_number'])
        Europe_countries = list((dataframe.groupby('region').get_group(country)['region']))
        divisionResults = dataframe.groupby('region').get_group(country)['deaths_weekly']\
            .div(dataframe.groupby('region').get_group(country)['population']).reset_index(drop=True)
        divisionResults = divisionResults * nr_people

        week_number = list(itertools.chain.from_iterable(week_nr))

        data = {
            'region' : Europe_countries,
            'week_number' : week_number[::-1],
            'death_rate' : divisionResults [::-1]
        }
        if pos == 0:
            death_rate_df = pd.DataFrame(data)

        if pos > 0:
            df = pd.DataFrame(data)
            death_rate_df = pd.concat([death_rate_df, df])
        
    return death_rate_df

In [260]:
death_rate = get_death_rate_per_inh(Europe2020 ,['Italy', 'Germany', 'Sweden', 'Greece'],100000)

In [261]:
death_rate

,region,week_number,death_rate
51,Italy,week_01,0.000000
50,Italy,week_02,0.000000
49,Italy,week_03,0.000000
48,Italy,week_04,0.000000
47,Italy,week_05,0.000000
...,...,...,...
4,Greece,week_48,6.359212
3,Greece,week_49,5.799751
2,Greece,week_50,5.100424
1,Greece,week_51,4.662179


In [262]:
plot_circ(death_rate,"deathrate in countries in Europe per Week", "death_rate","week_number","region", [0,360])

In [30]:
def comp_deathrate_germany(filename, death_rate_df, continent_df):
    with open(filename, 'r') as file:
        file_dict = pd.read_csv(file)
        sorted_deaths_per_week = file_dict['Value'].to_list()[::3]
        sorted_deaths_per_week.pop()
        sorted_deaths_per_week = [w.replace(',', '') for w in sorted_deaths_per_week]
        
        cov_death_rate_germany = death_rate_df.groupby('region').get_group('Germany').reset_index(drop=True)
        names = ['covid_deaths_germany'] * 52
        cov_death_rate_germany['region'] = names
        
        nor_deaths = continent_df[['region', 'week_number', 'population']].groupby('region').get_group('Germany').reset_index(drop=True)
        nor_deaths['deaths'] = sorted_deaths_per_week[::-1]
        nor_deaths['deaths'] =  pd.to_numeric(nor_deaths["deaths"])
        divisionResults = nor_deaths['deaths'].div(nor_deaths['population']).reset_index(drop=True)
        divisionResults = round(divisionResults * 100000, 1).to_list()
        nor_deaths['death_rate'] = divisionResults
        
        comp_death_rate = pd.concat([cov_death_rate_germany, nor_deaths[::-1]])
        
        return comp_death_rate[::-1] 

In [31]:
comp_death_rate = comp_deathrate_germany('demo_r_mwk_ts_1_Data.csv', death_rate, Europe2020)

In [32]:
plot_linear(comp_death_rate,"death rate comparison in Germany no normal death count 2020", "week_number", "death_rate", "region")